<h1>Processamento dos Dados Brutos

## Instalação e importação das bibliotecas necessárias

In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date
from pathlib import Path
import json

### 📌 Etapa 1: Processamento dos Perfis HTML e Extração de Informações

Com os perfis do LinkedIn salvos localmente em formato HTML, esta etapa tem como objetivo processar esses arquivos e extrair dados relevantes estruturados sobre **experiências profissionais** e **formação acadêmica**.

#### 🔍 Ações realizadas:

1. **Leitura dos Arquivos HTML**
   - Todos os arquivos na pasta `html_perfis` com extensão `.html` são carregados e processados individualmente utilizando a biblioteca `BeautifulSoup`.

2. **Extração de Informações Pessoais**
   - O nome do perfil é extraído da tag `<h1>`.
   - O título profissional (headline) é obtido de um `div` com classe `text-body-medium`.

3. **Extração de Seções Específicas**
   - As seções de **"Experience"** e **"Education"** são localizadas com base no conteúdo textual.
   - Para cada item dentro dessas seções, são extraídos:
     - **Título do cargo/curso**
     - **Empresa/instituição**
     - **Período**
     - **Descrição** (quando disponível)

4. **Tratamento e Estruturação dos Dados**
   - Os dados extraídos são organizados em uma estrutura de dicionário contendo os seguintes campos:
     - `nome`, `headline`, `cargo_curso`, `empresa_instituicao`, `periodo`, `descricao`, `tipo`, `arquivo`.

5. **Exportação para CSV**
   - Todos os registros coletados são armazenados em um `DataFrame` do pandas.
   - O resultado é exportado para o arquivo `perfil_linkedin_dados.csv`.

#### ✅ Resultado:
- Dados organizados de forma tabular.
- Cada linha do CSV representa uma experiência ou formação associada a um perfil processado.


In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup

PASTA = "../raw/html_perfis"
dados = []

def extrair_secao_por_texto(soup, chave):
    secoes = []
    for section in soup.find_all("section"):
        attrs_text = " ".join([str(val).lower() for val in section.attrs.values()])
        section_text = section.get_text(strip=True).lower()

        if chave.lower() in attrs_text or chave.lower() in section_text:
            for li in section.find_all("li"):
                linhas = li.get_text(separator="\n", strip=True).split("\n")
                linhas = [l.strip() for l in linhas if l.strip()]
                if len(linhas) < 2:
                    continue

                titulo_1 = linhas[0]
                subtitulo_1 = linhas[1]
                periodo = ""
                descricao = ""

                for l in linhas[2:]:
                    if any(m in l.lower() for m in ["de ", "to ", "até", "present", "moment", "–"]):
                        periodo = l
                    else:
                        descricao += l + " "

                secoes.append({
                    "titulo": titulo_1,
                    "subtitulo": subtitulo_1,
                    "periodo": periodo,
                    "descricao": descricao.strip()
                })
            break
    return secoes

for arquivo in os.listdir(PASTA):
    if not arquivo.endswith(".html"):
        continue

    caminho = os.path.join(PASTA, arquivo)
    with open(caminho, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")

    nome = ""
    headline = ""

    try:
        h1 = soup.find("h1")
        if h1:
            nome = h1.get_text(strip=True)
    except:
        pass

    try:
        headline_div = soup.find("div", class_="text-body-medium")
        if headline_div:
            headline = headline_div.get_text(strip=True)
    except:
        pass

    experiencias = extrair_secao_por_texto(soup, "Experience")
    educacoes = extrair_secao_por_texto(soup, "Education")

    # Juntar tudo só em um texto por tipo
    texto_exp = " ".join([
        f"{e['titulo']} - {e['subtitulo']} ({e['periodo']}) {e['descricao']} |||"
        for e in experiencias
    ])

    texto_edu = " ".join([
        f"{e['titulo']} - {e['subtitulo']} ({e['periodo']}) {e['descricao']} |||"
        for e in educacoes
    ])

    dados.append({
        "name": nome,
        "headline": headline,
        "experience": texto_exp.strip(),
        "education": texto_edu.strip(),
    })

if dados:
    df = pd.DataFrame(dados)
    df.to_csv("perfil_linkedin_dados.csv", index=False, encoding="utf-8")
    print(f"✅ {len(dados)} perfis salvos em perfil_linkedin_dados.csv")


✅ 557 perfis salvos em perfil_linkedin_dados.csv


In [ ]:
df_linkedin = pd.read_csv("perfil_linkedin_dados.csv")
df_linkedin

,name,headline,experience,education
0,Ana Beatriz De Souza Maciel,Aluno na Escola Superior de Tecnologia-UEA,Samsung Ocean - Samsung Ocean (- Fui responsáv...,Universidade do Estado do Amazonas - UEA - Uni...
1,Álef Monteiro,Angular | React | Django | Docker | PostgreSQL...,Full Stack Developer - Full Stack Developer (A...,Universidade do Estado do Amazonas - UEA - Uni...
2,Janderson Chaves C. Meireles,Software Developer | Electrical Engineering | ...,Sidia Instituto de Ciência e Tecnologia - Sidi...,Universidade do Estado do Amazonas - UEA - Uni...
3,Janderson Lira,Software Developer | Python | Machine Learning...,Top skills - Top skills (Python • Processament...,FGV - Fundação Getulio Vargas - FGV - Fundação...
4,Janielson Moura,Desenvolvedor Fullstack,"Venturus - Venturus (Análise de problemas, Exp...",Centro Universitário Fametro - Centro Universi...
...,...,...,...,...
552,Igor Rodrigues,Data Analyst | Data Scientist | Machine Learni...,Top skills - Top skills (SQL • Inteligência ar...,MBA USP/Esalq - MBA USP/Esalq (Analítica de da...
553,Isaac Felipe dos Santos Lima,Software Developer at Localiza&Co || Back-end ...,Software Developer - Software Developer (Feb 2...,Universidade do Estado do Amazonas - UEA - Uni...
554,Italo Thiago,Computers Engeniring student,Webmaster - Webmaster (Feb 2025 to Present · 5...,Universidade do Estado do Amazonas - UEA - Uni...
555,Izabelly Araújo,Talent Acquisition | Tech Recruiter | Human Re...,Top skills - Top skills () Communication • Neg...,IPOG - Instituto de Pós-Graduação e Graduação ...


### 📌 Etapa 2: Criação e Salvamento dos Metadados do Dataset

Nesta etapa, são definidos e salvos os metadados associados ao dataset processado a partir dos perfis do LinkedIn. Essa documentação é essencial para garantir rastreabilidade, reprodutibilidade e facilitar o entendimento e reutilização dos dados por outros usuários ou sistemas.

#### 🔍 Ações realizadas:

1. **Criação de Catálogo do Datalake**
   - Um objeto `datalake_catalog` foi definido com informações gerais do dataset, incluindo:
     - Nome do dataset
     - Descrição
     - Caminho para o arquivo CSV processado
     - Formato
     - Data de criação
     - Versão do conjunto de dados
   - O catálogo foi salvo como `datalake_linkedin_profile_data.json` na pasta `../metadata`.

2. **Mapeamento dos Tipos de Dados**
   - Foi implementada uma função (`map_dtype`) para mapear os tipos de dados do pandas para uma representação genérica (`string`, `integer`, `float`, etc.).
   - Essa função é usada para descrever cada coluna do dataset em termos do tipo de dado.

3. **Definição Detalhada de Metadados**
   - Os metadados incluem:
     - Nome do arquivo
     - Descrição do dataset
     - Formato, codificação e separador de campo
     - Número total de linhas
     - Lista de colunas com nome, tipo e descrição
     - Origem da coleta
     - Data da coleta
     - Responsáveis pelo projeto (`owner`)

4. **Armazenamento**
   - Os metadados completos foram salvos no arquivo `processed_data_metadata.json`, também dentro da pasta `../metadata`.

#### ✅ Resultado:
Dois arquivos JSON contendo os metadados do dataset:
- `datalake_linkedin_profile_data.json`: para fins de catálogo no datalake.
- `processed_data_metadata.json`: descrição técnica detalhada do conteúdo do dataset.


In [12]:
datalake_catalog = [
  {
    "dataset_name": "Linkedin Profiles Data",
    "description": "Dataset generated from LinkedIn profiles, containing professional experiences and education.",
    "path": "../processed/perfil_linkedin_dados.csv",
    "format": "csv",
    "created_at": str(date.today()),
    "version": "1.0"
  }
]

In [14]:
catalog_path = Path('../metadata')
catalog_path.mkdir(parents=True, exist_ok=True)
file_path = catalog_path / "datalake_linkedin_profile_data.json"
with open(file_path, "w", encoding="utf-8") as f:
  json.dump(datalake_catalog, f, ensure_ascii=False, indent=4)

In [35]:
def map_dtype(dtype):
    if pd.api.types.is_string_dtype(dtype):
        return "string"
    elif pd.api.types.is_integer_dtype(dtype):
        return "integer"
    elif pd.api.types.is_float_dtype(dtype):
        return "float"
    elif pd.api.types.is_bool_dtype(dtype):
        return "boolean"
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return "datetime"
    else:
        return "string"
    
columns_info = [
    {"name": col, "type": map_dtype(dtype), "description": "string"}
    for col, dtype in df_linkedin.dtypes.items()
]

columns_info

[{'name': 'name', 'type': 'string', 'description': 'string'},
 {'name': 'headline', 'type': 'string', 'description': 'string'},
 {'name': 'experience', 'type': 'string', 'description': 'string'},
 {'name': 'education', 'type': 'string', 'description': 'string'}]

In [36]:
metadados = {
  "file": "perfil_linkedin_dados.csv",
  "description": "Dataset containing professional experiences and education extracted from LinkedIn profiles.",
  "format": "csv",
  "separator": ",",
  "encoding": "utf-8",
  "number_of_rows": len(df_linkedin),
  "columns": 
    [{'name': 'name', 'type': 'string', 'description': 'Person name'},
      {'name': 'headline', 'type': 'string', 'description': 'Professional headline'},
      {'name': 'experience', 'type': 'string', 'description': 'Professional experiences'},
      {'name': 'education', 'type': 'string', 'description': 'Education details'}],
  "origin": "Unknown",
  "collect_date": "2025-06-19",
  "owner": "Ronald Boadana, Debora Barros and Arthur Santos"
}

In [37]:
metadata_path = "../metadata/processed_data_metadata.json"
with open(metadata_path, "w", encoding="utf-8") as f:
  json.dump(metadados, f, ensure_ascii=False, indent=4)